In [ ]:
# Leave for imports

Run the following !pip3 cells to install boto3 which is needed for this sandbox.

In [ ]:
!pip3 install boto3

The big take away here? When we script, its so much easier to through in an automation that'll do what we need rather that try to wire up libraries, search stack overflow, etc. 

# Automations and Scripting


A common use of python, and other languages, is to write quick scripts. But, it doesn't long for quick scripts to become permenent, or, quick scripts to become lengthy scripts. Worst of all, a "quick script" can too easily become an integral part of a legacy software package. 

Automations are beneficial for scripting as well! Whether reducing common expressions or interfacing with APIs, automations can make it simpler. 

In this sandbox, we'll use managing AWS EC2 virtual machines as an example. What do automations let you focus on? What you want to - creating, connecting, listing, terminating, etc. 

##### Note
This sandbox does require an AWS account to truly experience. Please have your AWS keys ready, they will remain private (unless you publish them to a public repo... don't do that...)

### Setup

First, we need to pass in our AWS keys. Typically, you would do this in a setup file, like Setup.py, but we've placed them here for ease. Clearly, the keys below are placeholders. Put in your AWS account keys to fully experience the sandbox. 

In [ ]:
#[AWSAccessKey 0123456789abcdefghij]
#[AWSSecretAccessKey 0123456789a/0123456/0123456789abcdefghij]

The next automation isn't essential, but we have it here for simplicity. If it was missing, the first AWS automation would create the client. 

In [ ]:
#[AWS.Client]

### Create Instance
First, let's create an AWS VM instance. Now, typically this requires jump through some hoops, but we'll automate the code to jump through those hoops for you. The automation will create code to save the .pem key and manage it for you. It will also save information within Arctic Fox concerning the instance's operating system. Please be aware, more code may be generated than you'd expect. 

Creating an instance does require a name, we pass it in as the first item. 

In [ ]:
#[AWS.Instance codespaceExample --create]

### Connect to the Instance

Using the same AWS.Instance automation, let's pass in the item --connect. This time, the automation will generate code to connect to the named instance. Also, you don't have to find the .pem, format it correctly, find the ip address, or discern the correct username. No, automations do all that for you. 

In [ ]:
#[AWS.Instance codespaceExample --connect]

### List Instances
Other things we want to do when scripting is see what resources are in use. We added the ability to list instances so we could easily see what VMs were running. 

Just listing the instances gives a bit too much information to digest...

In [ ]:
#[AWS.Instance --list]

So, we wanted to be able just to see the names of the instances

In [ ]:
#[AWS.Instance --list --name]

Or, the running state of those instances

In [ ]:
#[AWS.Instance --list --state]

### Terminate Instances
Lastly, one of the biggest things is taking down instances you no longer need. Following the previous patterns, we wanted to do this easily

In [ ]:
#[AWS.Instance codespaceExample --terminate]

And that's it! We made this example for AWS scripting, but hopefully you can see how automations are pertinent to a wide range of scripts. And now, check out the next sandbox!